# Emotion Detection Machine Learning Pipeline

## Import Library

In [1]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

## Mengubah format label

In [10]:
import pandas as pd

df = pd.read_csv('data/Emotion_classify_Data.csv')

label_mapping = {'anger': 0, 'fear': 1, 'joy': 2}
df['Emotion'] = df['Emotion'].map(label_mapping)
df.to_csv('data/Emotion_classify_Data.csv', index=False)


## Set Variable

In [2]:
PIPELINE_NAME = "sayyidan-i-pipeline"
SCHEMA_PIPELINE_NAME = "sayyidan-i-tfdv-schema"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

In [3]:
DATA_ROOT = "data"

In [4]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

# Data Ingestion menggunakan Example Gen


In [5]:
# split data 80:20
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2)
    ]))

example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [6]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# Data Validation menggunakan StatisticsGen

In [7]:
# summary statistics

statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples']
    )

interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [8]:
interactive_context.show(statistics_gen.outputs['statistics'])

## Membuat skema data menggunakan SchemaGen

In [9]:
# Data schema
schema_gen = SchemaGen( statistics = statistics_gen.outputs["statistics"])

interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [10]:
interactive_context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Comment',BYTES,required,,-
'Emotion',INT,required,,-


## Membuat Validator menggunakan ExampleValidator

In [11]:
#mengidentifikasi anomali
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [12]:
interactive_context.show(example_validator.outputs['anomalies'])

## Data Preprocessing


In [13]:

TRANSFORM_MODULE_FILE = 'emotion_transform.py'

In [16]:
%%writefile {TRANSFORM_MODULE_FILE}
import tensorflow as tf

LABEL_KEY = 'Emotion'
FEATURE_KEY = "Comment"

def transformed_name(key):
    #mengubah nama field yang telah di transform
    return key + '_xf'

def preprocessing_fn(inputs):
    """
    transform fitur agar lowercase dan label menjadi integer
    
    """
    outputs = {}
    
    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    
    

    
    return outputs

Overwriting emotion_transform.py


In [17]:
transform = Transform(
        examples=example_gen.outputs['examples'],
        schema=schema_gen.outputs['schema'],
        module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)

interactive_context.run(transform)

INFO:tensorflow:Assets written to: pipelines\sayyidan-i-pipeline\Transform\transform_graph\6\.temp_path\tftransform_tmp\2c42026c2e604c06a35da58bfe0953c0\assets


INFO:tensorflow:Assets written to: pipelines\sayyidan-i-pipeline\Transform\transform_graph\6\.temp_path\tftransform_tmp\2c42026c2e604c06a35da58bfe0953c0\assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 6
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## Training Model
Model dilatih menggunakan Arsitektur yang terdiri dari beberapa lapisan:
1. *vectorize_layer* adalah lapisan yang digunakan untuk mengubah teks menjadi representasi numerik. Proses ini melibatkan tokenisasi teks (mengganti kata-kata menjadi token), mengkonversi ke huruf kecil untuk memastikan konsistensi, dan menghapus tanda baca.

2. *embedding* adalah lapisan yang digunakan untuk mengubah indeks kata menjadi vektor nilai-nilai yang kontinyu. Ini memungkinkan representasi yang lebih kaya untuk kata-kata, di mana kata-kata yang serupa dalam konteks akan memiliki vektor yang mendekati satu sama lain dalam ruang vektor yang dibuat.

3. *globalaveragepooling1D*, yang berfungsi untuk merata-ratakan vektor-vektor embedding dari kata-kata dalam satu kalimat menjadi vektor tunggal. Pendekatan ini membantu dalam mengurangi dimensi data dan fokus pada fitur-fitur penting dari seluruh kalimat.

4.  *dense* dengan 64 neuron dan fungsi aktivasi relu membantu dalam mempelajari hubungan yang kompleks antara fitur-fitur yang diekstraksi dari teks.

5. *dense* dengan 32 neuron dan fungsi aktivasi relu bertujuan untuk mengekstraksi pola-pola lebih lanjut dari fitur yang diperoleh sebelumnya.

6. *dense* dengan 3 neuron dan fungsi aktivasi softmax adalah lapisan output yang memiliki 3 unit neuron (sesuai dengan jumlah kelas output yang diinginkan) dengan fungsi aktivasi softmax. Fungsi softmax mengonversi skor-skornya menjadi probabilitas distribusi probabilitas kelas yang berbeda, yang berguna untuk klasifikasi teks ke dalam salah satu dari tiga kategori yang mungkin.

In [18]:
TRAINER_MODULE_FILE = "emotion_trainer.py"

In [19]:
%%writefile {TRAINER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft 
from tensorflow.keras import layers
import os  
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs
 
LABEL_KEY = "Emotion"
FEATURE_KEY = "Comment"
 
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"
 
def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
 
 
def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs,
             batch_size=64)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL_KEY))
    return dataset

 
# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100
 
vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)
 
 
embedding_dim=16
def model_builder():
    """Build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation="relu")(x)
    outputs = layers.Dense(3, activation='softmax')(x)
    
    
    model = tf.keras.Model(inputs=inputs, outputs = outputs)
    
    model.compile(
        loss = 'sparse_categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.01),
        metrics=['accuracy']
    
    )
    
    # print(model)
    model.summary()
    return model 
 
 
def _get_serve_tf_examples_fn(model, tf_transform_output):
    
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        
        feature_spec = tf_transform_output.raw_feature_spec()
        
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using the transformed features
        return model(transformed_features)
        
    return serve_tf_examples_fn
    
def run_fn(fn_args: FnArgs) -> None:
    
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )
    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
    
    
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)]])
    
    # Build the model
    model = model_builder()
    
    
    # Train the model
    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            epochs=10)
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples'))
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting emotion_trainer.py


In [20]:
from tfx.proto import trainer_pb2

trainer  = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)
interactive_context.run(trainer)

Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Comment_xf (InputLayer)     [(None, 1)]               0         
                                                                 
 tf.reshape (TFOpLambda)     (None,)                   0         
                                                                 
 text_vectorization (TextVe  (None, 100)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 64)                1088  

INFO:tensorflow:Assets written to: pipelines\sayyidan-i-pipeline\Trainer\model\7\Format-Serving\assets


744/744 [==============================] - 10s 12ms/step - loss: 0.2860 - accuracy: 0.8553 - val_loss: 0.3864 - val_accuracy: 0.9057
Epoch 2/10
740/744 [============================>.] - ETA: 0s - loss: 0.0291 - accuracy: 0.9899
Epoch 2: val_accuracy did not improve from 0.90569
744/744 [==============================] - 6s 8ms/step - loss: 0.0290 - accuracy: 0.9899 - val_loss: 0.5078 - val_accuracy: 0.9006
Epoch 3/10
741/744 [============================>.] - ETA: 0s - loss: 0.0237 - accuracy: 0.9916
Epoch 3: val_accuracy did not improve from 0.90569
744/744 [==============================] - 7s 9ms/step - loss: 0.0238 - accuracy: 0.9916 - val_loss: 0.8109 - val_accuracy: 0.8675
Epoch 4/10
741/744 [============================>.] - ETA: 0s - loss: 0.0181 - accuracy: 0.9935
Epoch 4: val_accuracy did not improve from 0.90569
744/744 [==============================] - 7s 10ms/step - loss: 0.0181 - accuracy: 0.9935 - val_loss: 0.6765 - val_accuracy: 0.9023
Epoch 5/10
742/744 [============

...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\sayyidan-i-pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\sayyidan-i-pipeline\Trainer\model\7\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## Analisis dan Validasi Model

In [21]:
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
 
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [36]:
import tensorflow_model_analysis as tfma 
 
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='Emotion')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            
            tfma.MetricConfig(class_name='Precision'),
            tfma.MetricConfig(class_name='Recall'),
            tfma.MetricConfig(class_name='F1Score'),
            tfma.MetricConfig(class_name='CategoricalAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})))
        
        ])
    ]
 
)

In [37]:
from tfx.components import Evaluator
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 14
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [38]:
# Visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'accuracy': …

## Push model menggunakan Pusher

In [31]:
from tfx.components import Pusher 
from tfx.proto import pusher_pb2 
 
pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/emotion-detection-model'))
 
)

In [32]:
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 12
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## Mengecek Model

In [33]:
import requests
from pprint import PrettyPrinter
 
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8181/v1/models/emotion-detection-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '81'}]}
